In [5]:
import footprint.footprint_funcs as ff
import footprint.calc_footprint_FFP_climatology as myfootprint_s
import xarray as xr
import pandas as pd
import numpy as np
import os
import glob
import pyproj as proj
import cartopy.crs as ccrs
import importlib
import matplotlib.pyplot as plt
importlib.reload(ff)

<module 'footprint.footprint_funcs' from '/Users/miksch/git/triangle-method/tri_method_jupyter/footprint/footprint_funcs.py'>

In [7]:
flux_dir = '/Users/miksch/Thesis_Files/Processed/Eagle_Lake/'
tri_dir = '/Users/miksch/Thesis_Files/Processed/tri_method/'

#Read in flux footprint data

#Read in sigv data

def date_parse_sigv_17(doy,hr):
    '''
    Sigv date parser (pd.read_csv) for sigv files
    Change yr to year being processed 
    '''
    
    yr='2017'
    
    if '2400' in hr:
        hr = '000'
        return pd.datetime.strptime(f'{yr}{int(doy)+1}{int(hr):04}', '%Y%j%H%M')
    else:
        return pd.datetime.strptime(f'{yr}{doy}{int(hr):04}', '%Y%j%H%M')
    
def date_parse_sigv_18(doy,hr):
    '''
    Sigv date parser (pd.read_csv) for sigv files
    Change yr to year being processed 
    '''
    
    yr='2018'
    
    if '2400' in hr:
        hr = '000'
        return pd.datetime.strptime(f'{yr}{int(doy)+1}{int(hr):04}', '%Y%j%H%M')
    else:
        return pd.datetime.strptime(f'{yr}{doy}{int(hr):04}', '%Y%j%H%M')

sigv_17_file = os.path.join(flux_dir,'EL_17','cross_var','el17_crosswind.csv')
sigv_17 = pd.read_csv(sigv_17_file,delim_whitespace=True,parse_dates={'TIMESTAMP':[0,1]}, 
                      date_parser=date_parse_sigv_17, index_col=0, names=['DOY','hrmin','mag_v','sig_v'])

sigv_18_file = os.path.join(flux_dir,'EL_18','cross_var','el18_crosswind.csv')
sigv_18 = pd.read_csv(sigv_17_file,delim_whitespace=True,parse_dates={'TIMESTAMP':[0,1]}, 
                      date_parser=date_parse_sigv_18, index_col=0,  names=['DOY','hrmin','mag_v','sig_v'])

sigv = pd.concat([sigv_17, sigv_18])

#Read in flux data
flux = pd.read_csv(os.path.join(flux_dir,'closed_eb.csv'), index_col=[0], parse_dates=True)

data = pd.concat([flux,sigv],axis=1,sort=True)

#Get all landsat dates + times
overpass_dates = ['2017-06-04', '2017-06-20', '2017-10-10',
                  '2018-06-07', '2018-06-23', '2018-07-09', '2018-08-10', '2018-09-11', '2018-09-27', '2018-10-13' 
                 ]

overpass_times = [f'{day}T11:07' for day in overpass_dates]

#Interpolate data to overpass times, then change datetimeindex to dates
min_data = data.resample('T',label='right').interpolate()
min_data = min_data[min_data.index.isin(overpass_times)]
min_data.index = min_data.index - pd.Timedelta(hours=11,minutes=7)
min_data['ef_footprint_raw'] = None
min_data['ef_footprint'] = None

#Read in triangle method output
raster_str = 'ls8_sharp'
raster = xr.open_dataset(os.path.join(tri_dir,raster_str,f'{raster_str}_v03_test.nc'))

In [8]:
#Model parameters
zm_s = 2.78 #Measurement height [m]
h_c = 0.05 #Height of canopy [m]
d = (2./3.) * h_c
h_s = 2000. #Height of boundary layer [m]
min_data['ol'] = zm_s/min_data['z_L'] #Monin-Obukhov length [m]
dx = 3. #Model resolution [m]
origin_d = 600. #Model bounds distance from origin [m]
start_hr = 7
end_hr = 20
hours = np.arange(start_hr,end_hr+1) #np.ndarray of hours

#Convert local lat/lon to local UTM (Eagle Lake: UTM 12)
station_coord = (-112.050747,41.166695) #Eagle Lake lat/lon in WGS-84
in_proj = proj.Proj(init='EPSG:4326')
out_proj = proj.Proj(init='EPSG:32612')
(station_x,station_y) = proj.transform(in_proj,out_proj,*station_coord)

#Loop through each day in the dataframe
for date in min_data.index:
    
    #Subset dataframe to only values in day of year
    print(f'Date: {date}')
    temp_df = min_data[min_data.index == date]
    temp_raster = raster.sel(time=date)

    try:
        temp_line = temp_df.loc[temp_df.index == date,:]

        #Calculate footprint
        temp_ffp = myfootprint_s.FFP_climatology(domain=[-origin_d,origin_d,-origin_d,origin_d],dx=dx,dy=dx,
                                zm=zm_s-d, z0=h_c*.123, h=h_s, rs=None,
                                ol=temp_line['ol'].values,sigmav=temp_line['sig_v'].values,
                                ustar=temp_line['ustar'].values,
                                wind_dir=temp_line['wdir_ec'].values,
                                crop=0,fig=0,verbosity=0)

        f_2d = np.array(temp_ffp['fclim_2d'])
        x_2d = np.array(temp_ffp['x_2d']) + station_x
        y_2d = np.array(temp_ffp['y_2d']) + station_y
        
        f_2d = f_2d*dx**2

    except Exception as e:

        print(f'{t} footprint failed')

        continue
    
    #Mask out points that are below a % threshold (defaults to 90%)
    f_2d = ff.mask_fp_cutoff(f_2d)
    
    #Create kd tree and lookup dataframe from the imagery
    kd, combine_xy = ff.footprint_ckdtree(temp_raster)
        
    weighted_ef = ff.weight_raster(x_2d, y_2d, f_2d, combine_xy, kd)
    min_data.loc[date,'ef_footprint_raw'] = weighted_ef
    min_data.loc[date,'ef_footprint'] = weighted_ef/.9
    print(weighted_ef)

Date: 2017-06-04 00:00:00
0.6982490051659183
Date: 2017-06-20 00:00:00
0.7514454176695972
Date: 2017-10-10 00:00:00
0.7897253772370776
Date: 2018-06-07 00:00:00
0.7514045563357499
Date: 2018-06-23 00:00:00
0.7234870105255029
Date: 2018-07-09 00:00:00
0.81245583705452
Date: 2018-08-10 00:00:00
0.7279482744897575
Date: 2018-09-11 00:00:00
0.8201613409617845
Date: 2018-09-27 00:00:00
0.8054021186385109
Date: 2018-10-13 00:00:00
0.7919141313299405


In [9]:
min_data['ef_station'] = min_data['le_closed'] / (min_data['rn_avg'] - min_data['gflux_avg'])
min_data.to_csv(os.path.join(tri_dir, raster_str, f'{raster_str}_output_run2_test.csv'))